In [1]:
from transformers import pipeline

c:\Miniconda\envs\pytorch\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
classifier = pipeline("sentiment-analysis")
sentences = ["I've been waiting for a HuggingFace course my whole life.",
             "I hate this so much!"]
classifier(sentences)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598046541213989},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

#### Implementing the pipeline constitutents

In [3]:
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained (checkpoint)

In [4]:
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors= 'pt')
print(inputs)
#The output itself is a dictionary containing two keys, input_ids and attention_mask. input_ids contains two rows of integers 
# (one for each sentence)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [5]:
from transformers import AutoModel
model = AutoModel.from_pretrained(checkpoint)
# This architecture contains only the base Transformer module: given some inputs, it outputs what we’ll call hidden states.
# These Hidden states are inputs to another part of the model, known as the head.
# Different tasks could have been performed with the same architecture, but each of these tasks will have a different head 

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
outputs = model(**inputs) ## Two * means the inputs is a dictionary with variable no. of keys
print(outputs['last_hidden_state'].shape)

torch.Size([2, 16, 768])


The outputs of Transformer models behave like dictionaries. we can access the elements by attributes. The Model and Head can be selected together 
in Hugging face transformer library. A non-exhaustive list is as follows:
*Model (retrieve the hidden states)
*ForCausalLM                    i.e. AutoModelForCausalLM
*ForMaskedLM                    i.e. AutoModelForMakedLM
*ForMultipleChoice
*ForQuestionAnswering
*ForSequenceClassification
*ForTokenClassification

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs['logits'].shape)

torch.Size([2, 2])


##### Post Processing the Output

In [8]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward>)


In [19]:
# pass the output through softmax to get probabilities against logits
import torch
import numpy as np
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
# print(predictions.round())
tmp = np.round(predictions.detach().numpy(),3)
print(tmp)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward>)
tensor([[0., 1.],
        [1., 0.]], grad_fn=<RoundBackward>)
[[0.04  0.96 ]
 [0.999 0.001]]


In [20]:
print(model.config.id2label)

{0: 'NEGATIVE', 1: 'POSITIVE'}


#### Combining it all Together

In [21]:
sentences = ["I've been waiting for a HuggingFace course my whole life.",
             "This is exciting ", "That is fuckin Diabolical"]
checkpoint='distilbert-base-uncased-finetuned-sst-2-english'
tknzr = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
myinputs = tknzr(sentences, padding=True, truncation= True, return_tensors='pt')
outputs= model(**myinputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1 )
tmp_pred = np.round(predictions.detach().numpy(), 3)
print(tmp_pred)


[[0.04  0.96 ]
 [0.    1.   ]
 [0.999 0.001]]
